In [1]:
import cantools

# Load the DBC file
db = cantools.database.load_file(r"D:\EM\Excel_To_DBC_Conversion\UltraLite.dbc")

# Open a C header file to write
with open('ultralite_can_messages.h', 'w') as f:
    f.write('#ifndef CAN_MESSAGES_H\n')
    f.write('#define CAN_MESSAGES_H\n\n')
    f.write('#include <stdint.h>\n\n')

    # Write macros for message IDs
    for message in db.messages:
        f.write(f'#define {message.name.upper()}_ID 0x{message.frame_id:X}\n')
    
    f.write('\n')

    # Write C structures for messages
    for message in db.messages:
        f.write(f'typedef struct {{\n')
        for signal in message.signals:
            # Determine the appropriate C type based on signal length
            if signal.length <= 8:
                signal_type = 'uint8_t'
            elif signal.length <= 16:
                signal_type = 'uint16_t'
            elif signal.length <= 32:
                signal_type = 'uint32_t'
            else:
                signal_type = 'uint64_t'  # handle larger signals

            # Check for signed signals
            if signal.is_signed:
                signal_type = signal_type.replace('uint', 'int')
                
            # Write signal declaration with bit field length
            f.write(f'    {signal_type} {signal.name} : {signal.length};\n')
        f.write(f'}} {message.name}_t;\n\n')
    
    # Write function prototypes for encoding messages
    for message in db.messages:
        f.write(f'void encode_{message.name}(const {message.name}_t* msg, uint8_t* data);\n')
    
    f.write('\n')

    # Write function prototypes for decoding messages
    for message in db.messages:
        f.write(f'void decode_{message.name}(const uint8_t* data, {message.name}_t* msg);\n')
    
    f.write('\n')
    f.write('#endif // CAN_MESSAGES_H\n')

print("Conversion complete. Check the 'ultralite_can_messages.h' file.")


Conversion complete. Check the 'ultralite_can_messages.h' file.
